## Set up the environment

### Set up the imports.

In [ ]:
import asyncio
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAITextEmbedding
)
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.connectors.memory.astradb import AstraDBMemoryStore

### Populate the memory store.

In [ ]:
collection_id = "aboutMe"

async def populate_memory(memory: SemanticTextMemory) -> None:
    # Add some documents to the semantic memory
    await memory.save_information(collection_id, id="info1", text="My name is Andrea")
    await memory.save_information(collection_id, id="info2", text="I currently work as a tour guide")
    await memory.save_information(collection_id, id="info3", text="I've been living in Seattle since 2005")
    await memory.save_information(collection_id, id="info4", text="I visited France and Italy five times since 2015")
    await memory.save_information(collection_id, id="info5", text="My family is from New York")

### Search the populated memory store

In [ ]:
async def search_memory_examples(memory: SemanticTextMemory) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await memory.search("aboutMe", question)
        print(f"Answer: {result[0].text}\n")

### Create a chat application that uses the populated Astra DB Serverless vector store as context for your queries.

In [ ]:
async def setup_chat_with_memory(
    kernel: sk.Kernel,
    service_id: str,
) -> sk.KernelFunction:
    
    fact1 = "what is my name?"
    fact2 = "where do I live?"
    fact3 = "where's my family from?"
    fact4 = "where have I traveled?"
    fact5 = "what do I do for work?"

    sk_prompt = """
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if
    it does not have an answer.

    Information about me, from previous conversations:
    - {{$fact1}} {{recall $fact1}}
    - {{$fact2}} {{recall $fact2}}
    - {{$fact3}} {{recall $fact3}}
    - {{$fact4}} {{recall $fact4}}
    - {{$fact5}} {{recall $fact5}}

    Chat:
    {{$chat_history}}
    User: {{$user_input}}
    ChatBot: """.strip()

    prompt_template_config = PromptTemplateConfig(
        template=sk_prompt,
        execution_settings={
            service_id: kernel.get_service(service_id).get_prompt_execution_settings_class()(service_id=service_id)
        },
    )
    chat_func = kernel.create_function_from_prompt(
        function_name="chat_with_memory",
        plugin_name="chat",
        prompt_template_config=prompt_template_config,
    )
    return chat_func
    """
    chat_func = kernel.create_semantic_function(sk_prompt, max_tokens=200, temperature=0.8)

    context = kernel.create_new_context()
    context["fact1"] = "what is my name?"
    context["fact2"] = "where do I live?"
    context["fact3"] = "where's my family from?"
    context["fact4"] = "where have I traveled?"
    context["fact5"] = "what do I do for work?"

    context[sk.core_plugins.TextMemoryPlugin.COLLECTION_PARAM] = "aboutMe"
    context[sk.core_plugins.TextMemoryPlugin.RELEVANCE_PARAM] = "0.8"

    context["chat_history"] = ""

    return chat_func, context

    """

### Chat with the memory store.

In [24]:
async def chat(kernel: sk.Kernel, chat_func: sk.KernelFunction) -> bool:
    try:
        user_input = input("User:> ")
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False

    answer = await kernel.invoke(chat_func, request=user_input)

    print(f"ChatBot:> {answer}")
    return True

### Run the main function to perform the operations.

In [ ]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()

chat_service_id = "chat"
oai_chat_service = OpenAIChatCompletion(service_id=chat_service_id, ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)
embedding_gen = OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id)

kernel.add_service(oai_chat_service)
kernel.add_service(embedding_gen)

#original volatile memorystore instance
#kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
#kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "TextMemoryPlugin")

app_token, db_id, region, keyspace = sk.astradb_settings_from_dot_env()
astra_store = AstraDBMemoryStore(app_token, db_id, region, keyspace, 1536, "cosine")

memory = SemanticTextMemory(storage=astra_store, embeddings_generator=embedding_gen)
kernel.import_plugin_from_object(TextMemoryPlugin(memory), "TextMemoryPlugin")

In [ ]:
print("Populating memory...")
await populate_memory(memory)

In [ ]:
print("Asking questions... (manually)")
await search_memory_examples(memory)

In [25]:
print("Setting up a chat (with memory!)")
chat_func = await setup_chat_with_memory(kernel, chat_service_id)

Overwriting function "chat_with_memory" in collection


Setting up a chat (with memory!)


In [26]:
print("Begin chatting (type 'exit' to exit):\n")
chatting = True
while chatting:
    chatting = await chat(kernel, chat_func)

Begin chatting (type 'exit' to exit):



Variable `$: fact1` not found in the KernelArguments
Variable `$: fact1` not found in the KernelArguments
Error occurred while invoking function recall: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_text_embedding.OpenAITextEmbedding'> service failed to generate embeddings", BadRequestError('Error code: 400 - {\'error\': {\'message\': "\'$.input\' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", \'type\': \'invalid_request_error\', \'param\': None, \'code\': None}}'))
Variable `$: fact2` not found in the KernelArguments
Variable `$: fact2` not found in the KernelArguments
Error occurred while invoking function recall: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_text_embedding.OpenAITextEmbedding'> service failed to generate embeddings", BadRequestError('Error code: 400 - {\'error\': {\'message\': "\'$.input\' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 

ChatBot:> Hello! How can I assist you today?


Exiting chat...
